In [2]:
import time
import re
import requests
import numpy as np
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By # 언제까지 브라우저 접속을 기다려줄 지 결정
from selenium.webdriver.support.ui import WebDriverWait # By 모듈과 같이 자주 쓰인다.
from selenium.webdriver.support import expected_conditions as EC # 어떤 상태까지 기다리겠다(페이지 클릭이나 커서 이동)
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

from tqdm import tqdm

# 찾을 정보
기업이름,  마감날짜, 필요조건, 우대조건,  연봉정보, 주요 업무, 신입경력, 

In [3]:
def get_contents_dict(soup):
    
    # initialization
    title = np.nan
    companyname = np.nan
    duedate = np.nan
    job = np.nan
    salary = np.nan
    exp = np.nan
    maintask = np.nan
    requirement = np.nan
    prefered = np.nan
    location = np.nan
    content_list = []

    # 공고 제목
    title = soup.find_all('h1', class_ = 'ttl')[0].text.strip()
    # print(title)

    # 마감 날짜
    try: duedate = soup.find_all('li', class_ = 'calendar')[0].find_all('span',class_ = 'day_text')[-1].text.strip()
    except: duedate = soup.find_all('li', class_ = 'calendar')[0].find_all('span')[-1].text.strip()
    print(duedate)

    # 기업 이름
    companyname = soup.find_all('span', class_ = 'company_name')[0].text.strip()
    # print(companyname)

    # 직무
    job = soup.find('li', class_ = 'briefcase').find('span',class_ = 'cont').text.strip()
    # print(briefcase)

    # 급여
    salary = soup.find('li', class_ = 'money').find('span',class_ = 'cont').text.strip()
    # print(salary)

    # 신입/경력
    exp = soup.find('li', class_ = 'experience').find('span',class_ = 'cont').text.strip()
    # print(exp)

    # 주요 업무, 자격요건, 우대사항
    paragraph_all = soup.find_all('div', class_ = 'paragraph')
    for i in paragraph_all:
        if '주요업무' in i.text.strip():
            maintask = i.find('div', class_ = 'text').text.strip()
            # print('주요업무: ', maintask)

        elif '자격요건' in i.text.strip():
            requirement = i.find('div', class_ = 'text').text.strip()
            # print('자격요건:', requirement)

        elif '우대사항' in i.text.strip():
            prefered = i.find('div', class_ = 'text').text.strip()
            # print('우대사항:', prefered)
            
    # 위치
    for i in soup.find_all('div', class_ = 'line_text_list'):
        if '회사위치' in i.text.strip():
            location = i.find('p').text.strip()
            # print('위치:', location)

    return {'companyname': companyname, 'duedate': duedate, 'job': job, 'salary': salary, 'exp': exp, 'maintask': maintask, 'requirement': requirement, 'prefered': prefered, 'location': location}


# 스크래핑 시작

In [5]:
df = pd.DataFrame()

options = Options()
# options.add_argument("user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36")
# chrome_options.add_argument("--headless") # 브라우저가 화면에 뜨지 않고 내부적으로 실행
driver = webdriver.Chrome("./chromedriver_85.exe", options = options)

# 잡플래닛 URL 접속 및 페이지 넘김
for i in range(1, 100):
    print('===================PAGE{}=================='.format(i))
    driver.get('https://www.jobplanet.co.kr/job/search?q={}&page={}'.format('빅데이터',i))

    # 크롬 브라우저 내부 대기
    driver.implicitly_wait(5)

    #하나의 페이지 안에서 하나의 공고 클릭
    for r in tqdm(range(1,11)):
        # print('========{}번째 공고========'.format(r))

        # Click each job decscription
        driver.find_element_by_xpath('//*[@id="job_search_app"]/div/div[2]/div[1]/div[2]/ul/li[{}]/a'.format(r)).click()
        time.sleep(1)

        # Refresh source
        bs = BeautifulSoup(driver.page_source, 'html.parser')

        # Get contents
        dict_ = get_contents_dict(bs)
        df = df.append(dict_, ignore_index=True)

===================PAGE1==================
100%|██████████| 10/10 [00:11<00:00,  1.18s/it]상시채용
===================PAGE2==================

100%|██████████| 10/10 [00:13<00:00,  1.33s/it]2020. 09. 30
===================PAGE3==================

 60%|██████    | 6/10 [00:07<00:05,  1.35s/it]2021. 03. 03


WebDriverException: Message: chrome not reachable
  (Session info: chrome=85.0.4183.121)


In [8]:
df[]

,companyname,duedate,exp,job,location,maintask,prefered,requirement,salary
0,(주)디다이브,2020. 10. 05,경력무관,데이터분석,"서울 강남구 테헤란로20길 16, 금정빌딩 7층",- 광고 퍼포먼스 데이터를 빠르고 안정적으로 처리하기 위한 데이터 인프라 설계\n-...,- 광고플랫폼 데이터 처리 설계 및 개발 경험\n- AWS를 활용한 빅데이터 처리 ...,"- 데이터 인프라, 어플리케이션 개발 경험\n- 데이터 소스로부터 데이터 파이프라인...",회사내규에 따름
1,현대글로비스(주),2020. 11. 15,경력무관,"기획, 데이터분석",서울 강남구 테헤란로 301 삼정빌딩,· 빅데이터/AI 과제 기획 \n- 빅데이터/AI 과제 기획 및 발굴 \n- 현업 ...,NaN,· 빅데이터/AI 과제 기획 경력자 \n· 빅데이터 분석 경력자,회사내규에 따름
2,(주)한국정보기술단,2020. 11. 10,5년 이상,데이터분석,"서울 강남구 학동로11길 56, 2층 (논현동,백송빌딩)","- 빅데이터 분석 및 컨설팅, 플랫폼 구축 등",NaN,"- 기술사, 감리사, 정보시스템수석감리원 및 감리원 자격 소유자\n- IT컨설팅분야...",회사내규에 따름
3,엘지디스플레이(주),2020. 12. 31,경력무관,데이터분석,서울 영등포구 여의대로 128 LG트윈타워,"1. 수율 향상/ 품질 사고 Zero : 신속한 불량원인 규명, 불량 사전방지, 장...",NaN,아래 알고리즘 중 하나 이상에 대한 설계와 구현 역량을 갖춘 인원\n- 인공지능 기...,회사내규에 따름
4,네이버웹툰(주),2020. 10. 04,경력무관,소프트웨어엔지니어,"경기 성남시 분당구 분당내곡로 117, 9층","- 글로벌 NO.1 라인웹툰, 국내 1위 네이버웹툰, 웹소설, 시리즈, 시리즈on의...","- Kafka, hadoop, hive, spark, flume 등 대용량 처리 경...",- 대용량 데이터 설계에 대한 기본 지식과 모델링 경험이 있으신 분\n- Java ...,회사내규에 따름
5,(주)카카오엔터프라이즈,2020. 12. 31,경력무관,"데이터분석, 소프트웨어엔지니어, 웹개발","경기 성남시 분당구 판교역로 235, 3층",다음과 같은 업무를 하며 입사 후 업무 전환도 가능하니 1개 이상의 업무에 적합한 ...,"- 4년 이상의 분산 시스템 경험\n- kubernetes, docker 를 이용한...","- 전산/컴퓨터공학 전공\n- 탄탄한 관련 기본 지식 (운영체제, 알고리즘, 자료구...",회사내규에 따름
6,현대자동차(주),2021. 02. 28,경력무관,"데이터분석, 프로젝트매니저",서울 서초구 헌릉로 12,- 머신러닝/딥러닝 활용 생산설비 보전 기획 \n- 빅데이터 활용 내부 프로젝트 수행,NaN,"- 해외대 이공계 석/박사 학위 소지 및 취득 예정자, 또는 그에 준하는 경력자 (...",회사내규에 따름
7,(주)커스토머인사이트,2020. 10. 24,"신입, 경력무관",데이터분석,서울 송파구 송파대로 167 테라타워 A720,"- 정형 데이터 분석, 비정형 데이터 분석\n- 빅 데이터 솔루션 Hadoop 구현",- Open Source Hadoop Data Platform 경험자\n- Micr...,- 경력 : 신입 또는 경력 2 년 이상\n- 대학 졸업 또는 석사 졸업 이상\n-...,회사내규에 따름
8,흥국화재해상보험(주),2020. 09. 30,3년 이상,"데이터분석, 시장조사/분석","서울 종로구 새문안로 68 (신문로1가,흥국생명빌딩)",- 전사 빅데이터 전략 수립 및 실행\n- 빅데이터 플랫폼 전략 수립 및 구축\n-...,"- 생보사/손보사 현업 근무 경력자 우대\n- 사회조사분석사, SAS 인증 자격증 ...",- 4년제 대학 졸업 이상\n- 관련 직무경험 3년 이상 (대리~차장급)\n- 통계...,면접 후 결정
9,(주)케이티넥스알,상시채용,8년 이상,웹개발,"서울트별시 서초구 효령로 176, B동(방배동, KT방배빌딩)",1. Backend\n- 자사 빅데이터 솔루션 개발\n- 오픈소스 트래킹\n- Sp...,NaN,세부 자격요건 및 우대사항은 아래 링크를 확인해주세요!\n* http://nexr....,면접 후 결정
